In [1]:
import os
from deepface import DeepFace
import cv2
from tqdm import tqdm
import numpy as np
import time

2025-01-29 05:56:44.962145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738130204.986267 1616149 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738130204.992066 1616149 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 05:56:45.033090: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_name = "Facenet512"
detector_backend = "mtcnn"

In [3]:
embeddings = []
file_path = []

for dirpath, dirnames, filenames in os.walk("/home/playground/candidate_verification/images/Humans"):
   for file in tqdm(filenames[:20]):
      try:  
            img_path = f"{dirpath}/{file}"
            result =  DeepFace.represent(img_path = img_path, model_name="Facenet512", detector_backend="mtcnn")            
            for r in result:
                  if r: 
                     embeddings.append(r.get("embedding"))
                     file_path.append(img_path)      
      
      except Exception as e:
         print(e)
               



 45%|████▌     | 9/20 [00:15<00:20,  1.86s/it]

Face could not be detected in /home/playground/candidate_verification/images/Humans/1 (2011).jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


100%|██████████| 20/20 [00:44<00:00,  2.23s/it]


In [182]:
file_path

['/home/playground/candidate_verification/images/Humans/1 (4985).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (4707).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (6894).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (5339).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (6143).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (1544).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (6232).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (2672).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (428).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (1336).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (1401).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (664).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (3464).jpg',
 '/home/playground/candidate_verification/images/Humans/1 (29).jpg',
 '/home/pl

#### Qdrant

In [229]:
import nest_asyncio
nest_asyncio.apply()

from qdrant_client import AsyncQdrantClient, models
from qdrant_client.models import Distance, VectorParams

class QdrantConnection:
    
    def __init__(self) -> None:
        self.url = os.getenv('QDRANT_URL')
        self.port = os.getenv('QDRANT_PORT')
        self.client = AsyncQdrantClient(url="http://localhost/", port=6333, timeout=60)

    async def create_collection(self, collection_name):
        try:
            result = await self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=512, 
                    distance=Distance.COSINE  # Use COSINE for FaceNet512
                ),
                optimizers_config=models.OptimizersConfigDiff(
                    memmap_threshold=20000  # Use memory-mapped storage for large datasets
                ),
                hnsw_config=models.HnswConfigDiff(
                    m=32,  # Number of connections per node (balances speed & accuracy)
                    ef_construct=400,  # Improves recall during indexing
                    full_scan_threshold=10000,  # Avoid brute-force scans
                    on_disk=True  # Store index on disk for large-scale datasets
                ),
                quantization_config=models.ScalarQuantization(  # Optional: Enable quantization for memory efficiency
                    scalar=models.ScalarQuantizationConfig(
                        type="int8", quantile=0.99, always_ram=True
                    )
                )
                # payload_schema={  # Define metadata schema for filtering
                #     "person_id": models.PayloadSchemaType.KEYWORD,
                #     "name": models.PayloadSchemaType.TEXT,
                #     "age": models.PayloadSchemaType.INTEGER,
                #     "timestamp": models.PayloadSchemaType.INTEGER
                # }
            )
            return result
            
        except Exception as e:
            print(f"Error creating collection: {e}")

    async def delete_collection(self, collection_name):
        try:
            result = await self.client.delete_collection(collection_name=collection_name)
            return result
        except Exception as e:
            print(f"Error deleting collection: {e}")

    async def get_collection_info(self, collection_name):
        try:
            result = await self.client.get_collection(collection_name)
            return result    
        except Exception as e:
            print(f"Error getting collection info: {e}")

    async def batch_upload_points_to_collection(self, collection_name, payload):
        try:
            operation_info = await self.client.upsert(
                collection_name=collection_name,
                wait=True,
                points=models.Batch(
                    ids=payload.get('ids'),
                    vectors=payload.get('vectors'),
                    payloads=payload.get('payload')
                )
            )
            return operation_info
        except Exception as e:
            print(f"Error uploading points: {e}")
    
    async def query_collection(self, collection_name, query_vector: list):
        try:
            search_result = await self.client.query_points(
                    collection_name=collection_name,
                    query=query_vector,
                    with_payload=True,
                    limit=3
                )
            
            return search_result.points
        
        except Exception as e:
              print(f"Error querying collection: {e}")

In [230]:
qdrant_client = QdrantConnection()

In [ ]:
res = await qdrant_client.create_collection("face-recognition")

In [190]:
res = await qdrant_client.delete_collection("face-recognition")

In [232]:
res = await qdrant_client.get_collection_info("face-recognition")
print(res)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=19 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=512, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=32, ef_construct=400, full_scan_threshold=10000, max_indexing_threads=0, on_disk=True, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=20000, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=Scala

In [222]:
# emp = np.array(embeddings)
import uuid
payload = {
    'ids': [str(uuid.uuid4()) for _ in range(len(embeddings))],
    'vectors' : embeddings,
    'payload': [{"file_path": file} for file in file_path]
}

In [223]:
res = await qdrant_client.batch_upload_points_to_collection('face-recognition', payload)
print(res)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [224]:
test_image_path = "/home/playground/candidate_verification/images/Humans/1 (4985).jpg"
result =  DeepFace.represent(img_path = test_image_path, model_name="Facenet512", detector_backend="mtcnn")

In [225]:
test_vectors = result[0].get('embedding')

In [233]:
res = await qdrant_client.query_collection('face-recognition',test_vectors)

In [234]:
res

[ScoredPoint(id='05b58719-7eeb-415d-a546-bd24a299fbbf', version=0, score=0.99999994, payload={'file_path': '/home/playground/candidate_verification/images/Humans/1 (4985).jpg'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='4c89a6ed-8e8e-4632-8c4f-0267c8389003', version=0, score=0.33696622, payload={'file_path': '/home/playground/candidate_verification/images/Humans/1 (664).jpg'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='56b0b97c-ef45-42c5-9935-09d5f096f48f', version=0, score=0.3196198, payload={'file_path': '/home/playground/candidate_verification/images/Humans/1 (3700).jpg'}, vector=None, shard_key=None, order_value=None)]